In [1]:
import logging
import warnings
import datalink
import sys

warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s | %(message)s',
                    level=logging.DEBUG,
                    stream=sys.stdout)
log = logging.getLogger(__name__)

In [2]:
datalink.test_output()

INFO | logging from datalink


A data store class is generated by the link_factory method.

The user can then create instances of this class in a similar way to 

creating `namedtuple` objects from the `collections` module.

In [3]:
MyStore = datalink.link_factory(db_path='/tmp/test.db', table_name='data',
                                data_fields={'a': None, 'b': [2,4], 'c': 'a string'})

In [4]:
d = MyStore()
d.data

DEBUG | Creating database: /tmp/test.db
INFO | - db created at path: /tmp/test.db


{'a': None, 'b': [2, 4], 'c': 'a string'}

The first time a property in the store is changed from its default value,

the store is persisted in the SQL database linked to the class.

In [5]:
# Data accessible through field name
print(d.a)
d.b
d.c

None


[2, 4]

'a string'

In [6]:
d.a = 12
d.data

DEBUG | Call to _save_state.


{'a': 12, 'b': [2, 4], 'c': 'a string'}

In [7]:
# The update method can be used to set multiple field values 
# with only one update pushed to the SQL db.
d.update(a=14, c='a new string')
d.data

DEBUG | Call to _save_state.
DEBUG | Found uuid


{'a': 14, 'b': [2, 4], 'c': 'a new string'}

In [8]:
# Now let's get the uuid and load into a new instance from the SQL database.
my_uuid = d.uuid
my_uuid

'b3a3717b-a097-4d7b-8085-392482f69def'

In [9]:
del(d)
d2 = MyStore(datalink_uuid=my_uuid)

DEBUG | Found uuid
Loading


In [10]:
d2.data

{'a': 14, 'b': [2, 4], 'c': 'a new string'}

In [11]:
# Link to the entry in the SQL db is maintained.
d2.a = 16
d2.data

DEBUG | Call to _save_state.
DEBUG | Found uuid


{'a': 16, 'b': [2, 4], 'c': 'a new string'}